In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from supabase import create_client, Client
import psycopg2
from psycopg2 import sql

In [8]:
# creating supabase client
load_dotenv()

url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
db_url: str = os.getenv("DATABASE_URL")
client: Client = create_client(url, key)

In [ ]:
# getting the student data from csv file
data = pd.read_csv("../data/deliberation_postprocess.csv")

# matching column names
match_dict = {
    'Name': 'name',
    'Code': 'code',
    'Physique 1': 'phy1',
    'Moyenne_1': 'moyenne_a',
    'Credit_1': 'credit_a',
    'Algebre 1': 'algebre1',
    'Analyse 1': 'analyse1',
    'Moyenne_2': 'moyenne_b',
    'Credit_2': 'credit_b',
    'Algorithmique et Structure de Donné 1': 'asd1',
    'Structure Machine 1': 'sm1',
    'Moyenne_3': 'moyenne_c',
    'Credit_3': 'credit_c',
    'Langue Etranger 1': 'le1',
    'Expression Scientifique Terminologie': 'est',
    'Moyenne_4': 'moyenne_d',
    'Credit_4': 'credit_d',
    'Semestre Credit': 'semestre_credit',
    'Moyenne Semestre': 'moyenne_semestre',
    'Section': 'section'
}

# renaming
data = data.rename(columns=match_dict)

In [10]:


# sechema
table_schema = {
    "name": "text",
    "code": "text",
    "phy1": "float8",
    "moyenne_a": "float8",
    "credit_a": "int8",
    "algebre1": "float8",
    "analyse1": "float8",
    "moyenne_b": "float8",
    "credit_b": "int8",
    "asd1": "float8",
    "sm1": "float8",
    "moyenne_c": "float8",
    "credit_c": "int8",
    "le1": "float8",
    "est": "float8",
    "moyenne_d": "float8",
    "credit_d": "int8",
    "semestre_credit": "int8",
    "moyenne_semestre": "float8",
    "section": "float8"
}


# table name 
table_name = "analysis_student_grades_usthb"
columns = ", ".join([f"{col} {dtype}" for col, dtype in table_schema.items()])

# table creation query
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id UUID DEFAULT gen_random_uuid() PRIMARY KEY,
    {columns}
)
"""


In [11]:
conn_details = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "sslmode": os.getenv("DB_SSLMODE")
}



try:
    conn = psycopg2.connect(**conn_details)
    print("Connection successful!")

    cursor = conn.cursor()
    cursor.execute(
        sql.SQL("""
            SELECT EXISTS (
                SELECT 1 FROM information_schema.tables
                WHERE table_name = %s
            );
        """),
        [table_name]
    )

    table_exists = cursor.fetchone()[0]

    if table_exists:
        print(f"Table '{table_name}' already exists.")
    else:
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table '{table_name}' created successfully.")

    cursor.close()
    conn.close()

except Exception as e:
    print(f"Failed to connect or execute query: {e}")


Connection successful!
Table 'analysis_student_grades_usthb' already exists.


In [ ]:
# inserting data
i = 0
for row in data.to_dict(orient='records'):
    response = client.table("analysis_student_grades_usthb").insert(row).execute()
    if i % 100 == 0:
        print(f"Inserted {i} rows successfully")
    i += 1
print(f"{i} Rows inserted successfully")

Inserted 0 rows successfully
Inserted 100 rows successfully
Inserted 200 rows successfully
Inserted 300 rows successfully
Inserted 400 rows successfully
Inserted 500 rows successfully
Inserted 600 rows successfully
Inserted 700 rows successfully
Inserted 800 rows successfully
Inserted 900 rows successfully
Inserted 1000 rows successfully
Inserted 1100 rows successfully
Inserted 1200 rows successfully
Inserted 1300 rows successfully
Inserted 1400 rows successfully
